## Engineering Rare Categories

[Feature Engineering for Machine Learning Course](https://www.trainindata.com/p/feature-engineering-for-machine-learning)

Rare values are categories within a categorical variable that are present only in a small percentage of the observations. There is no rule of thumb to determine how small is a small percentage, but typically, any value below 5 % can be considered rare.

As we discussed in section 3 of the course, Infrequent labels are so few, that it is hard to derive reliable information from them. But more importantly, if you remember from section 3, infrequent labels tend to appear only on train set or only on the test set:

- If only on the train set, they may cause over-fitting
- If only on the test set, our machine learning model will not know how to score them

Therefore, to avoid this behaviour, we tend to group those into a new category called 'Rare' or 'Other'.

Rare labels can appear in low or highly cardinal variables. There is no rule of thumb to determine how many different labels are considered high cardinality. It depend as well on how many observations there are in the dataset. In a dataset with 1,000 observations, 100 labels may seem a lot, whereas in a dataset with 100,000 observations it may not be so high.

Highly cardinal variables tend to have many infrequent or rare categories, whereas low cardinal variables, may have only 1 or 2 rare labels.

### Note the following:

**Note that grouping infrequent labels or categories under a new category called 'Rare' or 'Other' is the common practice in machine learning for business.**

- Grouping categories into rare for variables that show low cardinality may or may not improve model performance, however, we tend to re-group them into a new category to smooth model deployment.

- Grouping categories into rare for variables with high cardinality, tends to improve model performance as well.


## In this demo:

We will learn how to re-group rare labels under a new category called rare, and compare the implications of this encoding in variables with:

- One predominant category
- A small number of categories
- High cardinality

For this demo, we will use the House Sale dataset. We will re-group variables using Feature-engine.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import RareLabelEncoder

## House Sale Price dataset

In [2]:
# let's load the house price dataset

data = pd.read_csv("../../Datasets/houseprice.csv")
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Important

The identification of rare labels should be done using only the training set, and then propagated to the test set. Rare labels should be identified in the training set only. In practice, what we will do is identify **non-rare labels**, and then any other label, either in the train or the test or future live data that is not in that list of **non-rare** labels, will be re=grouped into the new category.

For example, let's imagine that we have in the training set the variable 'city' with the labels 'London', 'Manchester' and 'Yorkshire'. 'Yorkshire' is present in less than 5% of the observations so we decide to re-group it in a new category called 'Rare'.

In the test set, we should also replace 'Yorkshire' by 'Rare', regardless of the percentage of observations for 'Yorkshire' in the test set. In addition, if in the test set we find the category 'Milton Keynes', that was not present in the training set, we should also replace that category by 'Rare'. On other words, all categories present in test set, not present in the list of **non-rare** categories derived from the training set, should be treated as rare values and re-grouped into 'Rare'.

In [3]:
# let's divide into train and test set

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=["SalePrice"], axis=1),  # predictors
    data.SalePrice,  # target
    test_size=0.3,
    random_state=0,
)

X_train.shape, X_test.shape

((1022, 80), (438, 80))

## Encoding Rare Labels with Feature-Engine

In [4]:
# Rare value encoder
rare_encoder = RareLabelEncoder(
    tol=0.05,  # minimal percentage to be considered non-rare
    n_categories=4,  # minimal number of categories the variable should have to re-cgroup rare categories
    variables=[
        "Neighborhood",
        "Exterior1st",
        "Exterior2nd",
        "MasVnrType",
        "ExterQual",
        "BsmtCond",
    ],  # variables to re-group
)

In [5]:
rare_encoder.fit(X_train.fillna("Missing"))

C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable MasVnrType is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable ExterQual is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


RareLabelEncoder(n_categories=4,
                 variables=['Neighborhood', 'Exterior1st', 'Exterior2nd',
                            'MasVnrType', 'ExterQual', 'BsmtCond'])

Note how the encoder is warning us that the variable **ExterQual** contains less than 4 categories, and thus, categories will not be regrouped under Rare, even if the percentage of observations is less than 0.05.

In [6]:
rare_encoder.variables_

['Neighborhood',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'BsmtCond']

In [7]:
# the encoder_dict_ is a dictionary of variable: frequent labels pair
rare_encoder.encoder_dict_

{'Neighborhood': ['NAmes',
  'CollgCr',
  'OldTown',
  'Edwards',
  'Sawyer',
  'Somerst',
  'Gilbert'],
 'Exterior1st': ['VinylSd', 'HdBoard', 'Wd Sdng', 'MetalSd', 'Plywood'],
 'Exterior2nd': ['VinylSd', 'Wd Sdng', 'HdBoard', 'MetalSd', 'Plywood'],
 'MasVnrType': ['BrkFace', 'Missing', 'Stone', 'BrkCmn'],
 'ExterQual': ['TA', 'Gd', 'Ex', 'Fa'],
 'BsmtCond': ['TA']}

In [8]:
X_train = rare_encoder.transform(X_train.fillna("Missing"))
X_test = rare_encoder.transform(X_test.fillna("Missing"))